<a href="https://colab.research.google.com/github/ar4857/Data_Analysis-/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow numpy matplotlib opencv-python tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model


In [ ]:
# Define paths (Modify these)
PAIRED_DIR = "/content/drive/MyDrive/GAN/paired_dataset_art"  # Path to paired images

IMG_SIZE = 256

def load_images_from_folder(folder):
    images = []
    files = sorted(os.listdir(folder))  # Sorting ensures correct pair matching
    for file in tqdm(files[:500]):  # Load first 500 images (adjust as needed)
        img = cv2.imread(os.path.join(folder, file))
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 127.5 - 1  # Normalize to [-1, 1]
            images.append(img)
    return np.array(images)

# Load only paired images (damaged & restored)
damaged_images = load_images_from_folder(os.path.join(PAIRED_DIR, "damaged"))
restored_images = load_images_from_folder(os.path.join(PAIRED_DIR, "undamaged"))

print(f"Loaded {len(damaged_images)} damaged images and {len(restored_images)} restored images.")


100%|██████████| 117/117 [00:04<00:00, 23.78it/s]


Loaded 116 damaged images and 117 restored images.


In [1]:
def build_generator():
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    def conv_block(x, filters, apply_batchnorm=True):
        x = Conv2D(filters, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
        if apply_batchnorm:
            x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        return x

    def deconv_block(x, skip_input, filters, dropout=False):
        x = Conv2DTranspose(filters, kernel_size=4, strides=2, padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        if dropout:
            x = Dropout(0.5)(x)
        x = Concatenate()([x, skip_input])
        x = ReLU()(x)
        return x

    down1 = conv_block(inputs, 64, apply_batchnorm=False)
    down2 = conv_block(down1, 128)
    down3 = conv_block(down2, 256)
    down4 = conv_block(down3, 512)
    down5 = conv_block(down4, 512)

    bottleneck = conv_block(down5, 512, apply_batchnorm=False)

    up1 = deconv_block(bottleneck, down5, 512, dropout=True)
    up2 = deconv_block(up1, down4, 512, dropout=True)
    up3 = deconv_block(up2, down3, 256)
    up4 = deconv_block(up3, down2, 128)
    up5 = deconv_block(up4, down1, 64)

    outputs = Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh')(up5)

    return Model(inputs, outputs)

generator = build_generator()
generator.summary()

In [3]:
def build_discriminator():
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    target = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    x = Concatenate()([inputs, target])

    down1 = Conv2D(64, 4, strides=2, padding='same')(x)
    down1 = LeakyReLU()(down1)

    down2 = Conv2D(128, 4, strides=2, padding='same')(down1)
    down2 = BatchNormalization()(down2)
    down2 = LeakyReLU()(down2)

    down3 = Conv2D(256, 4, strides=2, padding='same')(down2)
    down3 = BatchNormalization()(down3)
    down3 = LeakyReLU()(down3)

    down4 = Conv2D(512, 4, strides=2, padding='same')(down3)
    down4 = BatchNormalization()(down4)
    down4 = LeakyReLU()(down4)

    patch_output = Conv2D(1, 4, strides=1, padding='same', activation='sigmoid')(down4)

    return Model([inputs, target], patch_output)

discriminator = build_discriminator()
discriminator.summary()

NameError: name 'Input' is not defined

In [ ]:
class Pix2PixGAN(tf.keras.Model):
    def _init_(self, generator, discriminator, lambda_reconstruction=100):
        super(Pix2PixGAN, self)._init_()
        self.generator = generator
        self.discriminator = discriminator
        self.lambda_reconstruction = lambda_reconstruction

    def compile(self, gen_optimizer, disc_optimizer, loss_fn):
        super(Pix2PixGAN, self).compile()
        self.gen_optimizer = gen_optimizer
        self.disc_optimizer = disc_optimizer
        self.loss_fn = loss_fn

    def train_step(self, batch_data):
        input_image, target_image = batch_data

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_image = self.generator(input_image, training=True)

            real_output = self.discriminator([input_image, target_image], training=True)
            fake_output = self.discriminator([input_image, generated_image], training=True)

            gen_loss = self.loss_fn(target_image, generated_image)
            disc_loss = self.loss_fn(real_output, fake_output)

        gen_grads = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        disc_grads = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.gen_optimizer.apply_gradients(zip(gen_grads, self.generator.trainable_variables))
        self.disc_optimizer.apply_gradients(zip(disc_grads, self.discriminator.trainable_variables))

        return {"gen_loss": gen_loss, "disc_loss": disc_loss}

pix2pix = Pix2PixGAN(generator, discriminator)
pix2pix.compile(
    gen_optimizer=tf.keras.optimizers.Adam(2e-4, beta_1=0.5),
    disc_optimizer=tf.keras.optimizers.Adam(2e-4, beta_1=0.5),
    loss_fn=tf.keras.losses.MeanSquaredError()
)

In [ ]:
EPOCHS = 50
BATCH_SIZE = 16

dataset = tf.data.Dataset.from_tensor_slices((damaged_images, restored_images)).batch(BATCH_SIZE)
pix2pix.fit(dataset, epochs=EPOCHS)